In [ ]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers
import pickle

In [ ]:
aggregated_data = pickle.load( open( '../data/processed/20230304/aggregated_data', 'rb' ) )

In [ ]:
[train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca,
    train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca,
    train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca,
    train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca,
    train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca,
    train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca,
    train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca,
    train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca,
    train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca,
    train_vgg_raw_pca, valid_vgg_raw_pca, test_vgg_raw_pca,
    train_vgg_bg_pca, valid_vgg_bg_pca, test_vgg_bg_pca,
    train_vgg_fg_pca, valid_vgg_fg_pca, test_vgg_fg_pca,
    y_train, y_valid, y_test] = aggregated_data

In [ ]:
# # including YAMNet
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_embedding_raw_pca, train_vgg_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_embedding_raw_pca, valid_vgg_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_embedding_raw_pca, test_vgg_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_embedding_bg_pca, train_vgg_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_embedding_bg_pca, valid_vgg_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_embedding_bg_pca, test_vgg_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_embedding_fg_pca, train_vgg_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_embedding_fg_pca, valid_vgg_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_embedding_fg_pca, test_vgg_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [ ]:
# excluding YAMNet
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_vgg_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_vgg_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_vgg_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_vgg_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_vgg_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_vgg_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_vgg_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_vgg_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_vgg_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [14]:
train_data_fg.shape

(11933, 158)

In [15]:
train_data_all.shape

(11933, 474)

# Lazy predict

In [ ]:
# from lazypredict.Supervised import LazyClassifier
# clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_vgg_bg_pca, valid_vgg_bg_pca, y_train, y_valid)
# models

# Tensorflow parameter tuning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt


In [16]:
def build_model_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=150, step=10),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model



In [17]:
def build_model_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=500, step=50),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [20]:
tuner_raw = kt.BayesianOptimization(build_model_narrow,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_raw_0306_noYAMNet')

tuner_bg = kt.BayesianOptimization(build_model_narrow,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_bg_0306_noYAMNet')

tuner_fg = kt.BayesianOptimization(build_model_narrow,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_fg_0306_noYAMNet')

tuner_all = kt.BayesianOptimization(build_model_wide,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_all_0306_noYAMNet')

In [21]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 89 Complete [00h 00m 14s]
val_accuracy: 0.7028839588165283

Best val_accuracy So Far: 0.7216632962226868
Total elapsed time: 00h 42m 25s

Search: Running Trial #90

Value             |Best Value So Far |Hyperparameter
0.47199           |0.2               |dr
1                 |1                 |num_layers
10                |10                |units_0
relu              |relu              |activation
0.0001            |0.0001            |lr
10                |460               |units_1
10                |460               |units_2
10                |10                |units_3
10                |460               |units_4
460               |10                |units_5
10                |10                |units_6
10                |460               |units_7
460               |460               |units_8
460               |460               |units_9

Epoch 1/50
373/373 [==============================] - 1s 2ms/step - loss: 1.6545 - accuracy: 0.5191 - val_loss: 0.9684 - val_accuracy: 

In [ ]:
tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
tuner_raw.results_summary(1)

In [ ]:
tuner_bg.results_summary(1)